In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os



# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

2024-10-29 11:42:21.337707: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 11:42:21.397735: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [4]:
import keras_tuner

def build_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Input(shape=[28,28]))
    model.add(keras.layers.Flatten())

    for i in range(hp.Int("nLayers", min_value=1, max_value=3, step=1)):
        model.add(keras.layers.Dense(
                units=hp.Int("units", min_value=100, max_value=300, step=50),
                activation=hp.Choice("activation", ["relu", "sigmoid", "tanh"]))
        )
   
    model.add(keras.layers.Dense(10, activation="softmax"))
    

        
    model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(hp.Choice("learningRate", [0.001, 0.0001])),
              metrics=["accuracy"])
     
    return model


build_model(keras_tuner.HyperParameters())

<Sequential name=sequential, built=True>

In [8]:
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    overwrite=True,
    directory="/tmp/kerastuner",
    project_name="example",
)
tuner.search_space_summary()

tuner.search(X_train, y_train, validation_data=(X_valid, y_valid))

Trial 83 Complete [00h 00m 04s]
val_accuracy: 0.5306000113487244

Best val_accuracy So Far: 0.7829999923706055
Total elapsed time: 00h 04m 45s


In [7]:
# Get the top hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
display(best_hps)
# Build the model with the best hp.
final_model = build_model(best_hps[0])